In [1]:
import pandas as pd
import numpy as np
#from aspect import aspect_extract

## games

In [2]:
adj_mod = pd.read_csv('resource_paper/adjective-modifiers.csv', names=['adverb','weight'], header=None)
adj_mod.set_index('adverb', inplace = True)
adj_mod.head()

,weight
adverb,
above,2
absolutely,2
abundantly,2
acutely,2
amazingly,2


In [3]:
aspect_voc =pd.read_csv('resource_paper/aspects_videogames.csv', names=['noun'], header=None)
aspect_voc.index.name = 'aspect'
aspect_voc.head()

,noun
aspect,
art_style,appearance
art_style,appearances
art_style,art style
art_style,art styles
art_style,art-style


In [4]:
opinion_lexicon = pd.read_csv('resource_paper/lexicon_videogames.csv', names=['aspect', 'adj','polarity'], header=None)
opinion_lexicon.set_index('adj', inplace = True)
opinion_lexicon.head()

,aspect,polarity
adj,,
guest,art_style,0
basic,art_style,-1
visual,art_style,0
bold,art_style,0
shaded,art_style,-1


In [5]:
opinion_lexicon.aspect.nunique()

19

In [6]:
aspect_annot = pd.read_csv('resource_paper/annotations_voc_amazon_Video_Games_5.txt', sep='\t', names=['userID','itemID','noun','aspect', 'adjective','is_negated','adjective_modifier'], header=None)
#aspect_annot.reset_index(inplace=True)
#aspect_annot.rename(columns={"index":"userID"},inplace = True)
aspect_annot.head(10)

,userID,itemID,noun,aspect,adjective,is_negated,adjective_modifier
0,A2HD75EMZR8QLN,0700099867,rate,price,sure,0,NaN
1,A3UR8NLLY1ZHCX,0700099867,voices,sounds,even,0,NaN
2,A1DLMTOTHQ4AST,0700099867,graphics,graphics,magnificent,0,NaN
3,A1DLMTOTHQ4AST,0700099867,gameplay,gameplay,excellent,0,NaN
4,A1DLMTOTHQ4AST,0700099867,graphics,graphics,good,0,NaN
5,A1DLMTOTHQ4AST,0700099867,graphics,graphics,enough,0,NaN
6,A1DLMTOTHQ4AST,0700099867,graphics,graphics,higher,0,NaN
7,A1DLMTOTHQ4AST,0700099867,graphics,graphics,framerate,0,NaN
8,A361M14PU2GUEG,0700099867,graphics,graphics,perfect,0,NaN
9,A2UTRVO4FDCBH6,0700099867,graphics,graphics,overall,0,NaN


In [7]:
aspect_annot['userID'].nunique()

21962

In [8]:
aspect_annot.head()

,userID,itemID,noun,aspect,adjective,is_negated,adjective_modifier
0,A2HD75EMZR8QLN,0700099867,rate,price,sure,0,NaN
1,A3UR8NLLY1ZHCX,0700099867,voices,sounds,even,0,NaN
2,A1DLMTOTHQ4AST,0700099867,graphics,graphics,magnificent,0,NaN
3,A1DLMTOTHQ4AST,0700099867,gameplay,gameplay,excellent,0,NaN
4,A1DLMTOTHQ4AST,0700099867,graphics,graphics,good,0,NaN


In [9]:
df=pd.read_parquet('sent_orientation_games.parquet', engine='fastparquet', index=False)
#df.to_csv('sent_orn.csv')
df.head()

,aspect,adjective,is_negated,adjective_modifier,SO,adj_polarity,adj_mod,is_affirmative
0,price,sure,0,None,0,0,1,1
1,sounds,even,0,None,0,0,1,1
2,graphics,magnificent,0,None,0,0,1,1
3,gameplay,excellent,0,None,0,0,1,1
4,graphics,good,0,None,0,0,1,1


In [10]:
SO_result=df['SO'].tolist()
aspect_annot['SO']=SO_result
aspect_annot.rename(columns={'userID':'user','itemID':'item'},inplace = True)
aspect_annot.to_csv('aspect_with_SO_games.csv',index=False)

In [11]:
df = pd.read_csv('aspect_with_SO_games.csv')
df.head()

,user,item,noun,aspect,adjective,is_negated,adjective_modifier,SO
0,A2HD75EMZR8QLN,0700099867,rate,price,sure,0,NaN,0
1,A3UR8NLLY1ZHCX,0700099867,voices,sounds,even,0,NaN,0
2,A1DLMTOTHQ4AST,0700099867,graphics,graphics,magnificent,0,NaN,0
3,A1DLMTOTHQ4AST,0700099867,gameplay,gameplay,excellent,0,NaN,0
4,A1DLMTOTHQ4AST,0700099867,graphics,graphics,good,0,NaN,0


In [12]:
def groupby_count(df, group, count):
    game_count = pd.DataFrame()
    game_count['count'] = df.groupby(group)[count].count()
    return game_count


def prune(df, condition):     ## returns a dataframe that meet the given condition
    user_n = df.loc[df['count'] < condition ]
    return user_n


game_count = groupby_count(aspect_annot, 'user', 'item')


user_5 = prune(game_count, 5)

user_less_5 = user_5.index

pruned_data_5 = aspect_annot.set_index('user').drop(user_less_5)
pruned_data_5.reset_index(inplace = True)
pruned_data_5.drop(columns=['noun','adjective','is_negated','adjective_modifier'],inplace=True)

In [13]:
pruned_data_5.item.nunique()

10218

In [14]:
from aspect_rec import aspect_recom

In [15]:
item_list = ['B00KAI3KW2','0700099867','B00JXW6GE0', 'B00JQHU9RC']

In [16]:
AR = aspect_recom()
r = AR.fit(pruned_data_5)
l = AR.predict_for_user('A2HD75EMZR8QLN', item_list)
l

item      0700099867  6050036071  7100027950  8176503290  907843905X  \
item_sim         0.0    0.816497    0.819892    0.770154    0.840168   

item      9625990674  9861019731  9882155456  B000003SQQ  B000006OVE  ...  \
item_sim    0.866025    0.816497    0.793857    0.640513    0.673575  ...   

item      B00J06HWC4  B00J128FPA  B00J226358  B00J6DLPLK  B00J9P3KBS  \
item_sim    0.694365    0.800641    0.408248    0.516398    0.720577   

item      B00JM3R6M6  B00JQ8YH6A  B00JQHU9RC  B00JXW6GE0  B00KAI3KW2  
item_sim     0.63901      0.3849    0.783349    0.793857    0.673575  

[1 rows x 10218 columns]
item      B00KAI3KW2  0700099867  B00JXW6GE0  B00JQHU9RC
item_sim    0.673575         0.0    0.793857    0.783349


,item_sim
item,
B00KAI3KW2,0.673575
0700099867,0.000000
B00JXW6GE0,0.793857
B00JQHU9RC,0.783349


In [17]:
import lenskit
import lenskit.crossfold as xf
from  lenskit.crossfold import TTPair
from lenskit.crossfold import sample_users, SampleN
pairs_user = list(sample_users(pruned_data_5, 2, 7000, xf.SampleN(1)))


In [113]:
for p in pairs_user:
    uf=get_userfeature(p.train)
    print(p.train['user'].shape)

(428332,)
(428332,)


In [2]:
import spacy 
import neuralcoref 
nlp = spacy.load('en') 
neuralcoref.add_to_pipe(nlp)

In [7]:
doc=nlp('I have a dog. I love her.')

In [8]:
doc._.coref_clusters

[a dog: [a dog, her]]

corenlp for python: https://stanfordnlp.github.io/stanfordnlp/

models for treebank: https://stanfordnlp.github.io/stanfordnlp/models.html

coref: https://github.com/huggingface/neuralcoref

https://github.com/huggingface/neuralcoref/issues/189

http://ir.ii.uam.es/aspects/


In [39]:
aspect_annot.head(20)

,userID,itemID,noun,aspect,adjective,is_negated,adjective_modifier,SO
0,A2HD75EMZR8QLN,0700099867,rate,price,sure,0,NaN,
1,A3UR8NLLY1ZHCX,0700099867,voices,sounds,even,0,NaN,
2,A1DLMTOTHQ4AST,0700099867,graphics,graphics,magnificent,0,NaN,
3,A1DLMTOTHQ4AST,0700099867,gameplay,gameplay,excellent,0,NaN,
4,A1DLMTOTHQ4AST,0700099867,graphics,graphics,good,0,NaN,
5,A1DLMTOTHQ4AST,0700099867,graphics,graphics,enough,0,NaN,
6,A1DLMTOTHQ4AST,0700099867,graphics,graphics,higher,0,NaN,
7,A1DLMTOTHQ4AST,0700099867,graphics,graphics,framerate,0,NaN,
8,A361M14PU2GUEG,0700099867,graphics,graphics,perfect,0,NaN,
9,A2UTRVO4FDCBH6,0700099867,graphics,graphics,overall,0,NaN,


In [21]:
import json
import pandas as pd
import numpy as np
df = pd.read_json('resource_paper/reviews_Video_Games_5.json', lines=True)
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0700099867,"[8, 12]",1,Installing the game was a struggle (because of...,"07 9, 2012",A2HD75EMZR8QLN,123,Pay to unlock content? I don't think so.,1341792000
1,0700099867,"[0, 0]",4,If you like rally cars get this game you will ...,"06 30, 2013",A3UR8NLLY1ZHCX,"Alejandro Henao ""Electronic Junky""",Good rally game,1372550400
2,0700099867,"[0, 0]",1,1st shipment received a book instead of the ga...,"06 28, 2014",A1INA0F5CWW3J4,"Amazon Shopper ""Mr.Repsol""",Wrong key,1403913600
3,0700099867,"[7, 10]",3,"I got this version instead of the PS3 version,...","09 14, 2011",A1DLMTOTHQ4AST,ampgreen,"awesome game, if it did not crash frequently !!",1315958400
4,0700099867,"[2, 2]",4,I had Dirt 2 on Xbox 360 and it was an okay ga...,"06 14, 2011",A361M14PU2GUEG,"Angry Ryan ""Ryan A. Forrest""",DIRT 3,1308009600


In [22]:
az_videogames = df.drop(columns=['helpful','overall','reviewTime','reviewerName','summary','unixReviewTime'])

In [23]:
az_videogames.rename(columns={'asin':'itemID', 'reviewerID':'user', 'reviewText':'reviews' },inplace=True)
az_videogames.head()

,itemID,reviews,user
0,0700099867,Installing the game was a struggle (because of...,A2HD75EMZR8QLN
1,0700099867,If you like rally cars get this game you will ...,A3UR8NLLY1ZHCX
2,0700099867,1st shipment received a book instead of the ga...,A1INA0F5CWW3J4
3,0700099867,"I got this version instead of the PS3 version,...",A1DLMTOTHQ4AST
4,0700099867,I had Dirt 2 on Xbox 360 and it was an okay ga...,A361M14PU2GUEG


In [24]:
def groupby_count(df, group, count):
    game_count = pd.DataFrame()
    game_count['count'] = df.groupby(group)[count].count()
    return game_count
cn = groupby_count(az_videogames, 'itemID', 'user').sort_values(by='count')
cn.head()

,count
itemID,
B0013VD52U,5
B00005NIT9,5
B001AIL54G,5
B001AMO1D4,5
B001AVOJ5K,5


In [29]:
az_videogames.shape

(231780, 3)

In [45]:
import lenskit
import lenskit.crossfold as xf
from  lenskit.crossfold import TTPair
from lenskit.crossfold import sample_users, SampleN
pairs_user = list(sample_users(az_videogames, 2, 10000, xf.SampleN(1)))


In [46]:
for p in pairs_user:
    print(p.train)

            itemID                                            reviews  \
0       0700099867  Installing the game was a struggle (because of...   
1       0700099867  If you like rally cars get this game you will ...   
2       0700099867  1st shipment received a book instead of the ga...   
3       0700099867  I got this version instead of the PS3 version,...   
4       0700099867  I had Dirt 2 on Xbox 360 and it was an okay ga...   
5       0700099867  Overall this is a well done racing game, with ...   
6       0700099867  Loved playing Dirt 2 and I thought the graphic...   
7       0700099867  I can't tell you what a piece of dog**** this ...   
8       0700099867  I initially gave this one star because it was ...   
9       0700099867  I still haven't figured this one out. Did ever...   
10      0700099867  I'm not quite finished with the game's DiRT To...   
11      0700099867  I have been playing car racing games since the...   
12      0700099867  Dirt 3 on DVDi collect racing g

In [5]:
sample_data = az_videogames.head(10)

In [12]:
import stanfordnlp
import pandas as pd
nlp = stanfordnlp.Pipeline(lang='en', treebank ='en_gum')
doc = nlp('The multiplayer isnot the best but it works just ok')
dependency = doc.sentences[0].dependencies
depTable = pd.DataFrame(dependency, columns=['gov', 'rel', 'dep'])
depTable

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_gum', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum_tagger.pt', 'pretrain_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum.pretrain.pt', 'lang': 'en', 'shorthand': 'en_gum', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_gum', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum_parser.pt', 'pretrain_path': '/home/amifaraj/stanfordnlp_resources/en_gu

,gov,rel,dep
0,<Word index=2;text=multiplayer;lemma=multiplay...,det,<Word index=1;text=The;lemma=the;upos=DET;xpos...
1,<Word index=3;text=isnot;lemma=isnot;upos=VERB...,nsubj,<Word index=2;text=multiplayer;lemma=multiplay...
2,<Word index=0;text=ROOT>,root,<Word index=3;text=isnot;lemma=isnot;upos=VERB...
3,<Word index=5;text=best;lemma=good;upos=ADJ;xp...,det,<Word index=4;text=the;lemma=the;upos=DET;xpos...
4,<Word index=3;text=isnot;lemma=isnot;upos=VERB...,obj,<Word index=5;text=best;lemma=good;upos=ADJ;xp...
5,<Word index=8;text=works;lemma=work;upos=VERB;...,cc,<Word index=6;text=but;lemma=but;upos=CCONJ;xp...
6,<Word index=8;text=works;lemma=work;upos=VERB;...,nsubj,<Word index=7;text=it;lemma=it;upos=PRON;xpos=...
7,<Word index=3;text=isnot;lemma=isnot;upos=VERB...,conj,<Word index=8;text=works;lemma=work;upos=VERB;...
8,<Word index=10;text=ok;lemma=ok;upos=ADJ;xpos=...,advmod,<Word index=9;text=just;lemma=just;upos=ADV;xp...
9,<Word index=8;text=works;lemma=work;upos=VERB;...,xcomp,<Word index=10;text=ok;lemma=ok;upos=ADJ;xpos=...


In [8]:
from nltk import sent_tokenize
#sentences = sent_tokenize(az_videogames.reviews[1])
sentences = az_videogames.reviews[1].split('.')
from nltk.tokenize import word_tokenize

def process(text):
    tokens = word_tokenize(text)
    processed =[]
    for word in tokens:
        word = word.replace("n't", "not")
        word = word.replace("'", "")
        word = word.replace("#", "")
        word = word.replace(";", "")
        word = word.replace("&", "and")
        word = word.replace("'", "o")
        processed.append(word)
    processed = ' '.join(map(str, processed))    
    return processed

In [9]:
from aspect import aspect_extract
AE = aspect_extract()
A=[]
for txt in sentences:
    proc_txt = process(txt)
    print(proc_txt)
    res = AE.aspect_extraction(proc_txt)
    print(res)
    A.append(res)

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_gum', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum_tagger.pt', 'pretrain_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum.pretrain.pt', 'lang': 'en', 'shorthand': 'en_gum', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_gum', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/amifaraj/stanfordnlp_resources/en_gum_models/en_gum_parser.pt', 'pretrain_path': '/home/amifaraj/stanfordnlp_resources/en_gu

AssertionError: 

In [10]:
A

[[[('car', 'rally', '_', True)]],
 [[('@card@', 'European', '_', True)],
  [('party', 'huge', '_', True)],
  [('party', 'rally', '_', True)]],
 [[('Music', 'European', 'very', True), ('Music', 'voice', 'very', True)]],
 [[('multiplayer', 'good', '_', False)], [('it', 'ok', 'just', True)]]]